In [60]:
import numpy as np
import pandas as pd
import requests 
from bs4 import BeautifulSoup
from fake_useragent import UserAgent
import xml.etree.ElementTree as et

import xmltodict


import json

from IPython.display import display

In [61]:
def url_split(url):
    main_,para_1 = url.split("?")
    para_2 = para_1.split("&")
    para_3 = [d.replace("=",":").split(":") for d in para_2 if len(d.replace("=",":").split(":")) != 1]
    params_ = {}
    #print(para_3)
    for key,value in para_3:
        params_[key] = value
    return main_,params_

In [62]:
url_main = "http://apis.data.go.kr/B460014/foodBankInfoService2"

In [63]:
encode = "%2Fxc7f1IdclYG5tj6DIg1cU51nehX%2FzyMWvw8PpOvhKJqmfs8VwS2fnyJi03lk0J1UC73TJxpw6VBrT4Dk6rwkA%3D%3D"
decode = "/xc7f1IdclYG5tj6DIg1cU51nehX/zyMWvw8PpOvhKJqmfs8VwS2fnyJi03lk0J1UC73TJxpw6VBrT4Dk6rwkA=="

In [64]:
query_1 = {
    "getUserInfo": "이용자", "getFcltyGrpInfo" : "이용시설단체", "getCntrbtrInfo":"기부자",
    "getRceptStat": "접수현황", "getProvdStat": "제공현황",
    "getCnttgInfo":"물품현황","getSpctrInfo":"센터통계", "getPreferInfo":"선호물품"
}

In [65]:
query_1_2 = {
    value:key for key,value in query_1.items()
}

In [66]:
df_in = pd.read_excel(r"C:\Users\Samsung\Desktop\all\Project\23_VRPTW\Data\Korea\Query_Book.xlsx",sheet_name="all")
df_out = pd.read_excel(r"C:\Users\Samsung\Desktop\all\Project\23_VRPTW\Data\Korea\Query_Book.xlsx",sheet_name="out")
df_item = pd.read_excel(r"C:\Users\Samsung\Desktop\all\Project\23_VRPTW\Data\Korea\Query_Book.xlsx",sheet_name="items")
df_in.head()

,항목명(영문),항목명(국문),항목크기,항목구분,샘플데이터,항목설명,구분
0,serviceKey,인증키,100,1,인증키,공공데이터포털에서 발급받은 인증키,이용자
1,numOfRows,한 페이지 결과 수,4,1,2,한 페이지 결과 수,이용자
2,pageNo,페이지 번호,4,1,1,페이지 번호,이용자
3,dataType,응답자료형식,4,0,xml,요청자료형식(xml/json),이용자
4,stdrYm,기준년월,6,1,201702,요청 조회 날짜 (YYYYMM),이용자


In [67]:
def query_df_out(query_df,query,version=None):
    a = query_df[query_df["구분"]==query]
    if version =="all":
        b = a
    elif version == 1:
        b = a[query_df["항목구분"]==0]
    else:
        b = a[query_df["항목구분"]==1]
    return b

## Making dictionary

In [68]:
df_in2 = df_in[["항목명(영문)","항목명(국문)"]].drop_duplicates(subset=["항목명(영문)"]).reset_index(drop=True)
df_out2 = df_out[["항목명(영문)","항목명(국문)"]].drop_duplicates(subset=["항목명(영문)"]).reset_index(drop=True)

In [122]:
cate_ = {}
for row,col in df_in2.iterrows():
    key_ = col["항목명(영문)"].replace("\\","")
    value_  =col["항목명(국문)"].replace("\\","")
    cate_[key_] = value_

for row,col in df_out2.iterrows():
    key_ = col["항목명(영문)"].replace("\\","")
    value_  =col["항목명(국문)"].replace("\\","")
    cate_[key_] = value_

In [123]:
cate_

{'serviceKey': '인증키',
 'numOfRows': '한 페이지 결과 수',
 'pageNo': '페이지 번호',
 'dataType': '데이터 타입',
 'stdrYm': '기준년월',
 'areaCd': '지역코드',
 'unitySignguCd': '통합시군구코드',
 'spctrCd': '지원센터코드',
 'userSeccd': '이용자구분코드',
 'userClscd': '이용자분류코드',
 'happyTrgterYn': '행복e음대상자여부',
 'fcltySeNm': '시설유형명',
 'fcltyGrpClscd': '시설단체분류코드',
 'cntrbtrSeccd': '기부자구분코드',
 'cntrBplcKndcd': '기부사업장종류코드',
 'cntrbtrNm': '기부자명',
 'rceptSpctrCd': '접수지원센터코드',
 'foodYn': '식품여부',
 'cnttgLclasCd': '기부물품대분류코드',
 'cnttgMlsfcCd': '기부물품중분류코드',
 'distbTmlmtUseYn': '유통기한사용여부',
 'injryGoodsIntrcpYn': '유해상품차단여부',
 'spctrUncd': '지원센터단위코드',
 'spctrSecd': '지원센터구분코드',
 'spctrStscd': '지원센터상태코드',
 'operMbyLclasCd': '운영주체대분류코드',
 'operMbySclasCd': '운영주체소분류코드',
 'sttemntSeccd': '신고구분코드',
 'registDe': '등록일자',
 'preferCnttgClscd': '선호물품분류코드',
 'totalCount': '데이터 총 개수',
 'resultCode': '응답메시지 코드',
 'resultMsg': '응답메시지 내용',
 'useAmt': '이용금액',
 'useCo': '이용건수',
 'userCo': '이용자수',
 'fcltyGrpCo': '시설단체수',
 'cntrAmt': '기부금액',
 'cntrCo': '기부건수',
 'cn

# 물품현황

In [71]:
query_1

{'getUserInfo': '이용자',
 'getFcltyGrpInfo': '이용시설단체',
 'getCntrbtrInfo': '기부자',
 'getRceptStat': '접수현황',
 'getProvdStat': '제공현황',
 'getCnttgInfo': '물품현황',
 'getSpctrInfo': '센터통계',
 'getPreferInfo': '선호물품'}

In [72]:
url_test = url_main + "/" + "getCnttgInfo" + "?serviceKey=" + encode
url_test

'http://apis.data.go.kr/B460014/foodBankInfoService2/getCnttgInfo?serviceKey=%2Fxc7f1IdclYG5tj6DIg1cU51nehX%2FzyMWvw8PpOvhKJqmfs8VwS2fnyJi03lk0J1UC73TJxpw6VBrT4Dk6rwkA%3D%3D'

In [73]:
query_df_out(df_in,"물품현황","all")

,항목명(영문),항목명(국문),항목크기,항목구분,샘플데이터,항목설명,구분
54,serviceKey,인증키,100,1,인증키,공공데이터포털에서 발급받은 인증키,물품현황
55,numOfRows,한 페이지 결과 수,4,1,2,한 페이지 결과 수,물품현황
56,pageNo,페이지 번호,4,1,1,페이지 번호,물품현황
57,dataType,응답자료형식,4,0,xml,요청자료형식 (xml/json),물품현황
58,stdrYm,기준년월,6,1,201702,요청 조회 날짜 (YYYYMM),물품현황
59,distbTmlmtUseYn,유통기한사용여부,1,0,N,요청 유통기한사용여부 (Y/N),물품현황
60,injryGoodsIntrcpYn,유해상품차단여부,1,0,Y,요청 유해상품차단여부 (Y/N),물품현황
61,foodYn,식품여부,1,0,Y,요청 식품여부 (Y/N),물품현황
62,cnttgLclasCd,기부물품대분류코드,10,0,02000000(신선식품),요청 기부물품대분류 코드,물품현황
63,cnttgMlsfcCd,기부물품중분류코드,10,0,02010000(농산물),요청 기부물품중분류 코드,물품현황


In [74]:
query_df_out(df_out,"물품","all")

,항목명(영문),항목명(국문),항목크기,항목구분,샘플데이터,항목설명,구분
80,numOfRows,한 페이지 결과 수,4,1,2,한 페이지당 표출,물품
81,pageNo,페이지 번호,4,1,1,페이지 수,물품
82,totalCount,데이터 총 개수,10,1,105,데이터 총 개수,물품
83,resultCode,응답메시지 코드,2,1,0,응답 메시지코드,물품
84,resultMsg,응답메시지 내용,100,1,NORMAL SERVICE,응답 메시지 설명,물품
85,dataType,데이터 타입,4,1,XML,응답,물품
86,stdrYm,기준년월,6,1,201702,응답 조회 날짜 (YYYYMM),물품
87,distbTmlmtUseYn,유통기한사용여부,1,1,N,응답 유통기한사용여부 (Y/N),물품
88,injryGoodsIntrcpYn,유해상품차단여부,1,1,Y,응답 유해상품차단여부 (Y/N),물품
89,foodYn,식품여부,1,1,Y,응답 식품여부 (Y/N),물품


## Making Params

In [75]:
paras_ = {
        "stdrYm": 0,
    "numOfRows": 0,
    "pageNo": 0

}

In [76]:
paras_["stdrYm"] = 201701
paras_["numOfRows"] = 100
paras_["pageNo"] = 1

for key,value in paras_.items():
    url_test += "&" + key + "=" + str(value)

In [77]:
url_test

'http://apis.data.go.kr/B460014/foodBankInfoService2/getCnttgInfo?serviceKey=%2Fxc7f1IdclYG5tj6DIg1cU51nehX%2FzyMWvw8PpOvhKJqmfs8VwS2fnyJi03lk0J1UC73TJxpw6VBrT4Dk6rwkA%3D%3D&stdrYm=201701&numOfRows=100&pageNo=1'

In [78]:
html = requests.get(url_test).text

In [79]:
html

'<?xml version="1.0" encoding="UTF-8"?>\r\n<response><header><resultCode>00</resultCode><resultMsg>NORMAL SERVICE</resultMsg></header><body><dataType>XML</dataType><items><item><stdrYm>201701</stdrYm><distbTmlmtUseYn>N</distbTmlmtUseYn><injryGoodsIntrcpYn>N</injryGoodsIntrcpYn><foodYn>Y</foodYn><cnttgLclasCd>02000000</cnttgLclasCd><cnttgMlsfcCd>02830000</cnttgMlsfcCd><cnttgQy>0</cnttgQy><acntbkAmt>0</acntbkAmt></item><item><stdrYm>201701</stdrYm><distbTmlmtUseYn>N</distbTmlmtUseYn><injryGoodsIntrcpYn>N</injryGoodsIntrcpYn><foodYn>Y</foodYn><cnttgLclasCd>01000000</cnttgLclasCd><cnttgMlsfcCd>01010000</cnttgMlsfcCd><cnttgQy>0</cnttgQy><acntbkAmt>0</acntbkAmt></item><item><stdrYm>201701</stdrYm><distbTmlmtUseYn>N</distbTmlmtUseYn><injryGoodsIntrcpYn>N</injryGoodsIntrcpYn><foodYn>Y</foodYn><cnttgLclasCd>02000000</cnttgLclasCd><cnttgMlsfcCd>02880000</cnttgMlsfcCd><cnttgQy>0</cnttgQy><acntbkAmt>0</acntbkAmt></item><item><stdrYm>201701</stdrYm><distbTmlmtUseYn>N</distbTmlmtUseYn><injryGoodsInt

In [80]:
result = xmltodict.parse(html)
dictionay = json.loads(json.dumps(result))["response"]["body"]["items"]["item"]
out = pd.DataFrame(dictionay)
out

,stdrYm,distbTmlmtUseYn,injryGoodsIntrcpYn,foodYn,cnttgLclasCd,cnttgMlsfcCd,cnttgQy,acntbkAmt
0,201701,N,N,Y,02000000,02830000,0,0
1,201701,N,N,Y,01000000,01010000,0,0
2,201701,N,N,Y,02000000,02880000,0,0
3,201701,N,N,Y,01000000,01160000,21677,25298793.16
4,201701,N,N,Y,02000000,02990000,392,2438914.24
...,...,...,...,...,...,...,...,...
95,201701,Y,N,Y,02000000,02880000,0,0
96,201701,Y,N,Y,02000000,02990000,0,0
97,201701,Y,N,N,03000000,03010000,242,2375750
98,201701,Y,N,N,03000000,03020000,0,0


In [81]:
for col_ in out.columns:
    target_ = df_item[df_item["구분"]==col_]
    dic_tar = {col["코드"]:col["명"] for row,col in target_.iterrows()}
    col_list = []
    if len(target_)!=0:
        data_ = out[col_].map(lambda x: int(x))#.map(lambda x: x[1:] if x[0]=="0" else())
        data_2 = [dic_tar[i] for i in data_.values]
        out[col_] = data_2
    else:
        pass

In [83]:
out = out.rename(columns={col:cate_[col] for col in out.columns})
out

,기준년월,유통기한사용여부,유해상품차단여부,식품여부,기부물품대분류코드,기부물품중분류코드,기부물품수량,장부금액
0,201701,N,N,Y,신선식품,수산물(냉동),0,0
1,201701,N,N,Y,가공식품,조미료,0,0
2,201701,N,N,Y,신선식품,신선식품세트류,0,0
3,201701,N,N,Y,가공식품,과자류,21677,25298793.16
4,201701,N,N,Y,신선식품,기타신선식품,392,2438914.24
...,...,...,...,...,...,...,...,...
95,201701,Y,N,Y,신선식품,신선식품세트류,0,0
96,201701,Y,N,Y,신선식품,기타신선식품,0,0
97,201701,Y,N,N,일상용품,위생용품,242,2375750
98,201701,Y,N,N,일상용품,일용잡화,0,0


# 이용자

In [104]:
query_1

{'getUserInfo': '이용자',
 'getFcltyGrpInfo': '이용시설단체',
 'getCntrbtrInfo': '기부자',
 'getRceptStat': '접수현황',
 'getProvdStat': '제공현황',
 'getCnttgInfo': '물품현황',
 'getSpctrInfo': '센터통계',
 'getPreferInfo': '선호물품'}

In [105]:
url_test = url_main + "/" + "getUserInfo" + "?serviceKey=" + encode

In [106]:
paras_ = {
        "stdrYm": 0,
    "numOfRows": 0,
    "pageNo": 0

}
paras_["stdrYm"] = 201701
paras_["numOfRows"] = 100
paras_["pageNo"] = 1

for key,value in paras_.items():
    url_test += "&" + key + "=" + str(value)

In [107]:
html = requests.get(url_test).text
result = xmltodict.parse(html)
dictionay = json.loads(json.dumps(result))["response"]["body"]["items"]["item"]
out = pd.DataFrame(dictionay)

In [108]:
for col_ in out.columns:
    target_ = df_item[df_item["구분"]==col_]
    dic_tar = {col["코드"]:col["명"] for row,col in target_.iterrows()}
    col_list = []
    if len(target_)!=0:
        try:
            data_ = out[col_].map(lambda x: int(x))#.map(lambda x: x[1:] if x[0]=="0" else())
        except:
            data_ = out[col_]
        data_2 = [dic_tar[i] for i in data_.values]
        out[col_] = data_2
    else:
        pass

In [109]:
out = out.rename(columns={col:cate_[col] for col in out.columns})
out

,기준년월,지역코드,통합시군구코드,지원센터코드,이용자구분코드,이용자분류코드,행복e음대상자여부,이용금액,이용건수,이용자수
0,201701,서울,도봉구,서울시광역푸드뱅크,저소득층,외국인노동자,N,19180,5,1
1,201701,서울,도봉구,서울시광역푸드뱅크,저소득층,저소득가정,N,809678.55,243,37
2,201701,서울,도봉구,서울시광역푸드뱅크,저소득층,한부모가정,N,348464.97,94,16
3,201701,서울,도봉구,서울시광역푸드뱅크,저소득층,기타,N,607721.4,197,29
4,201701,서울,도봉구,서울시광역푸드뱅크,저소득층,노인부부가구,N,27657,7,1
...,...,...,...,...,...,...,...,...,...,...
95,201701,부산,영도구,영도구푸드뱅크,기초생활보장수급자,결식아동,N,4148,40,20
96,201701,부산,영도구,영도구푸드뱅크,기초생활보장수급자,다문화가정,N,6948.48,15,6
97,201701,부산,영도구,영도구푸드뱅크,기초생활보장수급자,독거어르신,N,165222.36,399,167
98,201701,부산,영도구,영도구푸드뱅크,기초생활보장수급자,소년소녀가장,N,414.8,4,2


# 이용시설단체

In [84]:
query_1

{'getUserInfo': '이용자',
 'getFcltyGrpInfo': '이용시설단체',
 'getCntrbtrInfo': '기부자',
 'getRceptStat': '접수현황',
 'getProvdStat': '제공현황',
 'getCnttgInfo': '물품현황',
 'getSpctrInfo': '센터통계',
 'getPreferInfo': '선호물품'}

In [85]:
url_test = url_main + "/" + "getFcltyGrpInfo" + "?serviceKey=" + encode
url_test

'http://apis.data.go.kr/B460014/foodBankInfoService2/getFcltyGrpInfo?serviceKey=%2Fxc7f1IdclYG5tj6DIg1cU51nehX%2FzyMWvw8PpOvhKJqmfs8VwS2fnyJi03lk0J1UC73TJxpw6VBrT4Dk6rwkA%3D%3D'

In [86]:
query_df_out(df_in,"이용시설단체","all")

,항목명(영문),항목명(국문),항목크기,항목구분,샘플데이터,항목설명,구분
11,serviceKey,인증키,100,1,인증키,공공데이터포털에서 발급받은 인증키,이용시설단체
12,numOfRows,한 페이지 결과 수,4,1,2,한 페이지 결과 수,이용시설단체
13,pageNo,페이지 번호,4,1,1,페이지 번호,이용시설단체
14,dataType,응답자료형식,4,0,xml,요청자료형식(xml/json),이용시설단체
15,stdrYm,기준년월,6,1,201702,요청 조회 날짜 (YYYYMM),이용시설단체
16,areaCd,지역코드,2,0,01(중앙),요청 시도코드 코드정보별첨 – 지역코드 정보 참조,이용시설단체
17,unitySignguCd,통합시군구코드,8,0,01000000(중앙),요청 통합시군구코드 코드정보별첨 – 통합시군구코드 정보 참조,이용시설단체
18,spctrCd,지원센터코드,6,0,S00101(전국푸드뱅크),요청 지원센터 코드 코드정보별첨 – 센터코드정보 참조,이용시설단체
19,fcltySeNm,시설유형명,50,0,이용시설,요청 시설유형명 코드정보별첨 – 시설유형명정보 참조,이용시설단체
20,fcltyGrpClscd,시설단체분류코드,2,0,11(무료급식소),요청 시설단체분류 코드 코드정보별첨 – 시설단체분류코드정보 참조,이용시설단체


In [87]:
query_df_out(df_out,"이용시설단체","all")

,항목명(영문),항목명(국문),항목크기,항목구분,샘플데이터,항목설명,구분
16,numOfRows,한 페이지 결과 수,4,1,2,한 페이지당 표출,이용시설단체
17,pageNo,페이지 번호,4,1,1,페이지 수,이용시설단체
18,totalCount,데이터 총 개수,10,1,2125,데이터 총 개수,이용시설단체
19,resultCode,응답메시지 코드,2,1,0,응답 메시지코드,이용시설단체
20,resultMsg,응답메시지 내용,100,1,NORMAL SERVICE,응답 메시지 설명,이용시설단체
21,dataType,데이터 타입,4,1,XML,응답,이용시설단체
22,stdrYm,기준년월,6,1,201702,응답 조회 날짜 (YYYYMM),이용시설단체
23,areaCd,지역코드,2,1,01(중앙),응답 지역코드 코드정보별첨 – 지역코드 정보 참조,이용시설단체
24,unitySignguCd,통합시군구코드,8,1,01000000(중앙),응답 통합시군구코드 코드정보별첨 – 통합시군구코드 정보 참조,이용시설단체
25,spctrCd,지원센터코드,6,1,S00101(전국푸드뱅크),응답 지원센터코드 코드정보별첨 – 센터코드정보 참조,이용시설단체


## Making Params

In [88]:
paras_ = {
        "stdrYm": 0,
    "numOfRows": 0,
    "pageNo": 0

}

In [89]:
paras_["stdrYm"] = 201701
paras_["numOfRows"] = 100
paras_["pageNo"] = 1

for key,value in paras_.items():
    url_test += "&" + key + "=" + str(value)

In [90]:
url_test

'http://apis.data.go.kr/B460014/foodBankInfoService2/getFcltyGrpInfo?serviceKey=%2Fxc7f1IdclYG5tj6DIg1cU51nehX%2FzyMWvw8PpOvhKJqmfs8VwS2fnyJi03lk0J1UC73TJxpw6VBrT4Dk6rwkA%3D%3D&stdrYm=201701&numOfRows=100&pageNo=1'

In [91]:
html = requests.get(url_test).text

In [92]:
html

'<?xml version="1.0" encoding="UTF-8"?>\r\n<response><header><resultCode>00</resultCode><resultMsg>NORMAL SERVICE</resultMsg></header><body><dataType>XML</dataType><items><item><stdrYm>201701</stdrYm><areaCd>02</areaCd><unitySignguCd>02000019</unitySignguCd><spctrCd>S00301</spctrCd><fcltySeNm>이용시설</fcltySeNm><fcltyGrpClscd>15</fcltyGrpClscd><useAmt>35737138.77</useAmt><useCo>45</useCo><fcltyGrpCo>4</fcltyGrpCo></item><item><stdrYm>201701</stdrYm><areaCd>02</areaCd><unitySignguCd>02000019</unitySignguCd><spctrCd>S00301</spctrCd><fcltySeNm>기타시설</fcltySeNm><fcltyGrpClscd>52</fcltyGrpClscd><useAmt>4653191.11</useAmt><useCo>10</useCo><fcltyGrpCo>1</fcltyGrpCo></item><item><stdrYm>201701</stdrYm><areaCd>02</areaCd><unitySignguCd>02000019</unitySignguCd><spctrCd>S00301</spctrCd><fcltySeNm>기타시설</fcltySeNm><fcltyGrpClscd>53</fcltyGrpClscd><useAmt>3830000</useAmt><useCo>629</useCo><fcltyGrpCo>1</fcltyGrpCo></item><item><stdrYm>201701</stdrYm><areaCd>02</areaCd><unitySignguCd>02000019</unitySigng

In [99]:
result = xmltodict.parse(html)
dictionay = json.loads(json.dumps(result))["response"]["body"]["items"]["item"]
out = pd.DataFrame(dictionay)
out

,stdrYm,areaCd,unitySignguCd,spctrCd,fcltySeNm,fcltyGrpClscd,useAmt,useCo,fcltyGrpCo
0,201701,02,02000019,S00301,이용시설,15,35737138.77,45,4
1,201701,02,02000019,S00301,기타시설,52,4653191.11,10,1
2,201701,02,02000019,S00301,기타시설,53,3830000,629,1
3,201701,02,02000019,S00301,생활시설,33,20542525.95,120,6
4,201701,02,02000019,S00301,생활시설,37,6454605.81,16,2
...,...,...,...,...,...,...,...,...,...
95,201701,02,02000018,S01501,생활시설,42,264661.56,1,1
96,201701,02,02000018,S01501,이용시설,11,2443394.27,6,4
97,201701,02,02000018,S01501,이용시설,14,61127019.68,272,28
98,201701,02,02000018,S01501,생활시설,37,1058646.24,4,4


In [100]:
for col_ in out.columns:
    target_ = df_item[df_item["구분"]==col_]
    dic_tar = {col["코드"]:col["명"] for row,col in target_.iterrows()}
    col_list = []
    if len(target_)!=0:
        try:
            data_ = out[col_].map(lambda x: int(x))#.map(lambda x: x[1:] if x[0]=="0" else())
        except:
            data_ = out[col_]
        data_2 = [dic_tar[i] for i in data_.values]
        out[col_] = data_2
    else:
        pass

In [102]:
out = out.rename(columns={col:cate_[col] for col in out.columns})
out

,기준년월,지역코드,통합시군구코드,지원센터코드,시설유형명,시설단체분류코드,이용금액,이용건수,시설단체수
0,201701,서울,영등포구,영등포사랑나눔푸드뱅크,이용시설,기타이용시설,35737138.77,45,4
1,201701,서울,영등포구,영등포사랑나눔푸드뱅크,기타시설,종교사회단체,4653191.11,10,1
2,201701,서울,영등포구,영등포사랑나눔푸드뱅크,기타시설,기타,3830000,629,1
3,201701,서울,영등포구,영등포사랑나눔푸드뱅크,생활시설,부랑인시설,20542525.95,120,6
4,201701,서울,영등포구,영등포사랑나눔푸드뱅크,생활시설,장애인시설,6454605.81,16,2
...,...,...,...,...,...,...,...,...,...
95,201701,서울,금천구,금천구푸드뱅크,생활시설,기타생활시설,264661.56,1,1
96,201701,서울,금천구,금천구푸드뱅크,이용시설,무료급식소,2443394.27,6,4
97,201701,서울,금천구,금천구푸드뱅크,이용시설,지역아동센터,61127019.68,272,28
98,201701,서울,금천구,금천구푸드뱅크,생활시설,장애인시설,1058646.24,4,4


# 기부자

In [110]:
query_1

{'getUserInfo': '이용자',
 'getFcltyGrpInfo': '이용시설단체',
 'getCntrbtrInfo': '기부자',
 'getRceptStat': '접수현황',
 'getProvdStat': '제공현황',
 'getCnttgInfo': '물품현황',
 'getSpctrInfo': '센터통계',
 'getPreferInfo': '선호물품'}

In [112]:
url_test = url_main + "/" + "getCntrbtrInfo" + "?serviceKey=" + encode

In [113]:
paras_ = {
        "stdrYm": 0,
    "numOfRows": 0,
    "pageNo": 0

}
paras_["stdrYm"] = 201701
paras_["numOfRows"] = 100
paras_["pageNo"] = 1

for key,value in paras_.items():
    url_test += "&" + key + "=" + str(value)

In [125]:
html = requests.get(url_test).text
result = xmltodict.parse(html)
dictionay = json.loads(json.dumps(result))["response"]["body"]["items"]["item"]
out = pd.DataFrame(dictionay)

In [126]:
for col_ in out.columns:
    target_ = df_item[df_item["구분"]==col_]
    dic_tar = {col["코드"]:col["명"] for row,col in target_.iterrows()}
    col_list = []
    if len(target_)!=0:
        try:
            data_ = out[col_].map(lambda x: int(x))#.map(lambda x: x[1:] if x[0]=="0" else())
        except:
            data_ = out[col_]
        data_2 = [dic_tar[i] for i in data_.values]
        out[col_] = data_2
    else:
        pass

In [127]:
out = out.rename(columns={col:cate_[col] for col in out.columns})
out

,기준년월,지역코드,통합시군구코드,지원센터코드,기부자구분코드,기부사업장종류코드,기부자명,기부금액,기부건수,기부자수
0,201701,중앙,중앙,전국푸드뱅크,기업(개인사업자),"식품 도,소매업",맥케인인터내셔널인크 한국지점,30024000,1,1
1,201701,중앙,중앙,전국푸드뱅크,기업(개인사업자),"식품 도,소매업",산마을 영농조합법인,7500000,2,1
2,201701,중앙,중앙,전국푸드뱅크,기업(개인사업자),"식품 도,소매업",주식회사 그린티앤지인터내셔날,1320000,1,1
3,201701,중앙,중앙,전국푸드뱅크,기업(개인사업자),"식품 도,소매업",주식회사 제이엠푸드,62448750,2,1
4,201701,중앙,중앙,전국푸드뱅크,기업(개인사업자),"식품 도,소매업",지에스리테일,11356636.3,915,1
...,...,...,...,...,...,...,...,...,...,...
95,201701,서울,영등포구,영등포사랑나눔푸드뱅크,기업(개인사업자),"식품 도,소매업",아티제여의도롯데캐슬점,127000,11,1
96,201701,서울,영등포구,영등포사랑나눔푸드뱅크,기업(개인사업자),"식품 도,소매업",일신상사(CJ대리점),85400,1,1
97,201701,서울,영등포구,영등포사랑나눔푸드뱅크,기업(개인사업자),"식품 도,소매업",자연드림베이커리,695545,22,1
98,201701,서울,영등포구,영등포사랑나눔푸드뱅크,기업(개인사업자),"식품 도,소매업",파리바게뜨 영등포대우점,849533,22,1


# 접수현황

In [128]:
query_1

{'getUserInfo': '이용자',
 'getFcltyGrpInfo': '이용시설단체',
 'getCntrbtrInfo': '기부자',
 'getRceptStat': '접수현황',
 'getProvdStat': '제공현황',
 'getCnttgInfo': '물품현황',
 'getSpctrInfo': '센터통계',
 'getPreferInfo': '선호물품'}

In [129]:
url_test = url_main + "/" + "getRceptStat" + "?serviceKey=" + encode

In [130]:
paras_ = {
        "stdrYm": 0,
    "numOfRows": 0,
    "pageNo": 0

}
paras_["stdrYm"] = 201701
paras_["numOfRows"] = 100
paras_["pageNo"] = 1

for key,value in paras_.items():
    url_test += "&" + key + "=" + str(value)

In [131]:
html = requests.get(url_test).text
result = xmltodict.parse(html)
dictionay = json.loads(json.dumps(result))["response"]["body"]["items"]["item"]
out = pd.DataFrame(dictionay)

In [132]:
for col_ in out.columns:
    target_ = df_item[df_item["구분"]==col_]
    dic_tar = {col["코드"]:col["명"] for row,col in target_.iterrows()}
    col_list = []
    if len(target_)!=0:
        try:
            data_ = out[col_].map(lambda x: int(x))#.map(lambda x: x[1:] if x[0]=="0" else())
        except:
            data_ = out[col_]
        data_2 = [dic_tar[i] for i in data_.values]
        out[col_] = data_2
    else:
        pass

In [133]:
out = out.rename(columns={col:cate_[col] for col in out.columns})
out

,기준년월,지역코드,통합시군구코드,접수지원센터코드,식품여부,기부물품대분류코드,기부물품중분류코드,장부금액,장부수량,기부자수,기부건수
0,201701,중앙,중앙,S00101,N,일상용품,위생용품,3904,2,1,2
1,201701,중앙,중앙,S00101,N,일상용품,화장품,162929020,75706,2,2
2,201701,중앙,중앙,S00101,N,가정용품,생활용품,31308736,21929,3,4
3,201701,중앙,중앙,S00101,N,의약품/의료용품,의약품,2772,6,1,2
4,201701,중앙,중앙,S00101,Y,가공식품,조미료,22899489.16,10698,4,115
...,...,...,...,...,...,...,...,...,...,...,...
95,201701,서울,성동구,S35601,Y,가공식품,유제품,488000,476,4,4
96,201701,서울,성동구,S35601,Y,가공식품,축산가공식품,756000,28,7,7
97,201701,서울,성동구,S35601,Y,가공식품,수산가공식품,80000,50,1,1
98,201701,서울,성동구,S35601,Y,가공식품,대용식기타,896500,136,2,2


# 제공현황

In [134]:
query_1

{'getUserInfo': '이용자',
 'getFcltyGrpInfo': '이용시설단체',
 'getCntrbtrInfo': '기부자',
 'getRceptStat': '접수현황',
 'getProvdStat': '제공현황',
 'getCnttgInfo': '물품현황',
 'getSpctrInfo': '센터통계',
 'getPreferInfo': '선호물품'}

In [135]:
url_test = url_main + "/" + "getProvdStat" + "?serviceKey=" + encode

In [136]:
paras_ = {
        "stdrYm": 0,
    "numOfRows": 0,
    "pageNo": 0

}
paras_["stdrYm"] = 201701
paras_["numOfRows"] = 100
paras_["pageNo"] = 1

for key,value in paras_.items():
    url_test += "&" + key + "=" + str(value)

In [137]:
html = requests.get(url_test).text
result = xmltodict.parse(html)
dictionay = json.loads(json.dumps(result))["response"]["body"]["items"]["item"]
out = pd.DataFrame(dictionay)

In [138]:
for col_ in out.columns:
    target_ = df_item[df_item["구분"]==col_]
    dic_tar = {col["코드"]:col["명"] for row,col in target_.iterrows()}
    col_list = []
    if len(target_)!=0:
        try:
            data_ = out[col_].map(lambda x: int(x))#.map(lambda x: x[1:] if x[0]=="0" else())
        except:
            data_ = out[col_]
        data_2 = [dic_tar[i] for i in data_.values]
        out[col_] = data_2
    else:
        pass

In [139]:
out = out.rename(columns={col:cate_[col] for col in out.columns})
out

,기준년월,지역코드,통합시군구코드,지원센터코드,식품여부,기부물품대분류코드,기부물품중분류코드,이용자수,제공건수,제공금액
0,201701,서울,도봉구,서울시광역푸드뱅크,Y,가공식품,분말류,142,142,367612
1,201701,서울,도봉구,서울시광역푸드뱅크,Y,가공식품,즉석/편의식품,495,606,1523640
2,201701,서울,도봉구,서울시광역푸드뱅크,Y,가공식품,음료류,252,252,208648.44
3,201701,서울,도봉구,서울시광역푸드뱅크,Y,신선식품,농산물,118,118,418900
4,201701,서울,도봉구,서울시광역푸드뱅크,Y,신선식품,수산물,70,70,215450
...,...,...,...,...,...,...,...,...,...,...
95,201701,서울,구로구,구로희망푸드뱅크,Y,가공식품,가공식품세트류,1,1,152000
96,201701,서울,구로구,구로희망푸드뱅크,Y,가공식품,음료류,16,16,2419077.6
97,201701,서울,구로구,구로희망푸드뱅크,N,가정용품,세탁용품,6,6,0
98,201701,서울,구로구,구로희망푸드뱅크,Y,가공식품,과자류,11,11,5837551.28


# 센터통계

In [140]:
query_1

{'getUserInfo': '이용자',
 'getFcltyGrpInfo': '이용시설단체',
 'getCntrbtrInfo': '기부자',
 'getRceptStat': '접수현황',
 'getProvdStat': '제공현황',
 'getCnttgInfo': '물품현황',
 'getSpctrInfo': '센터통계',
 'getPreferInfo': '선호물품'}

In [142]:
url_test = url_main + "/" + "getSpctrInfo" + "?serviceKey=" + encode

In [143]:
paras_ = {
        "stdrYm": 0,
    "numOfRows": 0,
    "pageNo": 0

}
paras_["stdrYm"] = 201701
paras_["numOfRows"] = 100
paras_["pageNo"] = 1

for key,value in paras_.items():
    url_test += "&" + key + "=" + str(value)

In [156]:
html = requests.get(url_test).text
result = xmltodict.parse(html)
dictionay = json.loads(json.dumps(result))["response"]["body"]["items"]["item"]
out = pd.DataFrame(dictionay)

In [157]:
out.head()

,stdrYm,areaCd,unitySignguCd,spctrUncd,spctrSecd,spctrCd,spctrTelno,spctrAdres,spctrDetailAdres,spctrStscd,operMbyLclasCd,operMbySclasCd,sttemntSeccd,registDe,rceptAmt,provdAmt,undtakeAmt,trnsferAmt,userCo
0,201701,01,01000000,01,20,S00101,02-2077-3985,서울특별시 마포구 만리재로 14,한국사회복지회관 4층,1,01,0101,02,20020520,1502818406.3,0,0,1518214594,0
1,201701,02,02000010,02,20,S00201,02-905-1377,서울 도봉구 창동,1-7,1,01,0101,02,20001205,214050197,14366322.44,81396821,665502574,652
2,201701,02,02000019,03,20,S00301,02-846-1377,서울특별시 영등포구 도림로61길 7(대림동),7,1,01,0101,02,20090101,184096211,131834005.5,2825560,5288924,190
3,201701,02,02000004,03,20,S00401,02-425-1377,서울특별시 성동구 왕십리로28길 10 준희빌딩(도선동),1층,1,03,0301,02,20020101,31325960,38783216.15,6467190,21876559,32
4,201701,02,02000001,03,20,S00501,02-762-1377,서울 종로구 숭인1동,55-2 숭인1동주민센터 4층,1,01,0101,01,20020101,70196990,62632106.18,24167245,32693612,38


In [158]:
for col_ in out.columns:
    target_ = df_item[df_item["구분"]==col_]
    dic_tar = {col["코드"]:col["명"] for row,col in target_.iterrows()}
    col_list = []
    if len(target_)!=0:
        try:
            data_ = out[col_].map(lambda x: int(x))#.map(lambda x: x[1:] if x[0]=="0" else())
        except:
            data_ = out[col_]
        try:
            data_2 = [dic_tar[i] for i in data_.values]
        except:
            pass
        out[col_] = data_2
    else:
        pass

areaCd
unitySignguCd
spctrUncd
spctrSecd
spctrCd
spctrStscd
operMbyLclasCd
operMbySclasCd
sttemntSeccd


In [159]:
out = out.rename(columns={col:cate_[col] for col in out.columns})
out

,기준년월,지역코드,통합시군구코드,지원센터단위코드,지원센터구분코드,지원센터코드,지원센터전화번호,지원센터주소,지원센터상세주소,지원센터상태코드,운영주체대분류코드,운영주체소분류코드,신고구분코드,등록일자,접수금액,제공금액,인수금액,이관금액,이용자수
0,201701,중앙,중앙,전국,푸드뱅크,전국푸드뱅크,02-2077-3985,서울특별시 마포구 만리재로 14,한국사회복지회관 4층,사용,법인 운영,사회복지법인,당연신고,20020520,1502818406.3,0,0,1518214594,0
1,201701,서울,서울,광역,푸드뱅크,서울시광역푸드뱅크,02-905-1377,서울 도봉구 창동,1-7,사용,법인 운영,사회복지법인,당연신고,20001205,214050197,14366322.44,81396821,665502574,652
2,201701,서울,서울,기초,푸드뱅크,영등포사랑나눔푸드뱅크,02-846-1377,서울특별시 영등포구 도림로61길 7(대림동),7,사용,법인 운영,사회복지법인,당연신고,20090101,184096211,131834005.5,2825560,5288924,190
3,201701,서울,서울,기초,푸드뱅크,성동희망푸드뱅크,02-425-1377,서울특별시 성동구 왕십리로28길 10 준희빌딩(도선동),1층,사용,비영리 민간 단체·시설 운영,비영리 민간 단체·시설,당연신고,20020101,31325960,38783216.15,6467190,21876559,32
4,201701,서울,서울,기초,푸드뱅크,종로구푸드뱅크,02-762-1377,서울 종로구 숭인1동,55-2 숭인1동주민센터 4층,사용,법인 운영,사회복지법인,임의신고,20020101,70196990,62632106.18,24167245,32693612,38
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,201701,경기,경기,기초,푸드뱅크,안성시푸드뱅크,031-674-9451,경기도 안성시 중앙로451번길 11,(봉남동) 한마음복지관 2,사용,법인 운영,사회복지법인,당연신고,20020101,58789603,68471256.65,12751020,23218862,437
96,201701,경기,경기,기초,푸드뱅크,구리시푸드뱅크,031-556-8100,경기 구리시 토평동,984번지,사용,비영리 민간 단체·시설 운영,비영리 민간 단체·시설,임의신고,20020101,18006023,18006021.66,1406108,0,899
97,201701,강원,강원,광역,푸드뱅크,강원도광역푸드뱅크,033-646-0197,강원도 강릉시 난설헌로 12-6,(운정동)사랑관 1층,사용,법인 운영,사회복지법인,당연신고,20100901,12271560,0,104880909,86810748,0
98,201701,강원,강원,기초,푸드뱅크,춘천시푸드뱅크,033-262-3494,강원 춘천시 중앙로 104-1,"3층, 춘천시사회복지협의회",사용,법인 운영,사회복지법인,당연신고,20031101,21539139,28901146.1,4879523,0,221


# 선호물품

In [160]:
query_1

{'getUserInfo': '이용자',
 'getFcltyGrpInfo': '이용시설단체',
 'getCntrbtrInfo': '기부자',
 'getRceptStat': '접수현황',
 'getProvdStat': '제공현황',
 'getCnttgInfo': '물품현황',
 'getSpctrInfo': '센터통계',
 'getPreferInfo': '선호물품'}

In [161]:
url_test = url_main + "/" + "getPreferInfo" + "?serviceKey=" + encode

In [162]:
paras_ = {
        "stdrYm": 0,
    "numOfRows": 0,
    "pageNo": 0

}
paras_["stdrYm"] = 201701
paras_["numOfRows"] = 100
paras_["pageNo"] = 1

for key,value in paras_.items():
    url_test += "&" + key + "=" + str(value)

In [163]:
html = requests.get(url_test).text
result = xmltodict.parse(html)
dictionay = json.loads(json.dumps(result))["response"]["body"]["items"]["item"]
out = pd.DataFrame(dictionay)

In [164]:
out.head()

,areaCd,unitySignguCd,spctrCd,preferCnttgClscd,holdQy
0,01,02000014,S00101,01,0
1,01,02000014,S00101,02,0
2,01,02000014,S00101,03,0
3,01,02000014,S00101,04,0
4,01,02000014,S00101,05,0


In [165]:
for col_ in out.columns:
    target_ = df_item[df_item["구분"]==col_]
    dic_tar = {col["코드"]:col["명"] for row,col in target_.iterrows()}
    col_list = []
    if len(target_)!=0:
        try:
            data_ = out[col_].map(lambda x: int(x))#.map(lambda x: x[1:] if x[0]=="0" else())
        except:
            data_ = out[col_]
        try:
            data_2 = [dic_tar[i] for i in data_.values]
        except:
            pass
        out[col_] = data_2
    else:
        pass

In [166]:
out = out.rename(columns={col:cate_[col] for col in out.columns})
out

,지역코드,통합시군구코드,지원센터코드,선호물품분류코드,보유수량
0,중앙,마포구,전국푸드뱅크,라면류,0
1,중앙,마포구,전국푸드뱅크,쌀(곡식류),0
2,중앙,마포구,전국푸드뱅크,밀가루(류),0
3,중앙,마포구,전국푸드뱅크,고추장(류),0
4,중앙,마포구,전국푸드뱅크,된장(류),0
...,...,...,...,...,...
95,서울,성동구,성동희망푸드나눔센터,간장(류),0
96,서울,성동구,성동희망푸드나눔센터,소면류(국수),4
97,서울,성동구,성동희망푸드나눔센터,김(류),0
98,서울,성동구,성동희망푸드나눔센터,즉석밥류,0
